In [ ]:
# 필요한 것 다운로드하는 코드
!pip install pyside6 pyinstaller selenium webdriver-manager beautifulsoup4 requests openpyxl pandas numpy

In [ ]:
pyinstaller -w -F --add-data "budongsan.ui;." --add-data "budongsan_icon.png;." --add-data "NanumGothic.ttf;." --add-data "NanumGothicBold.ttf;." --add-data "NanumGothicExtraBold.ttf;." --add-data "NanumGothicLight.ttf;." --exclude-module PyQt5 "삼삼엠투GUI프로그램_최종.py"


In [ ]:
# 프로그램 다운로드 코드
# pyinstaller -w -F

In [ ]:
import os
import time
import logging
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

def setup_logger():
    logging.basicConfig(
        level=logging.INFO,  # INFO 레벨로 설정
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.StreamHandler(),  # 콘솔 출력
            logging.FileHandler("pagination_test.log")  # 로그 파일
        ]
    )

setup_logger()

# 크롬 드라이버 옵션 설정
options = webdriver.ChromeOptions()
# options.add_argument('--headless')  # 브라우저 창을 띄우지 않으려면 주석 해제
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')

# webdriver-manager를 사용하여 크롬 드라이버 초기화
service = Service(ChromeDriverManager().install())
service.log_path = os.devnull  # webdriver-manager 로그 비활성화

driver = webdriver.Chrome(service=service, options=options)

def click_page_button(driver, nth_child):
    try:
        page_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, f".pagination > a:nth-child({nth_child})"))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", page_button)
        time.sleep(0.5)
        page_button.click()
        # 페이지 로드 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".room_item"))
        )
        time.sleep(2)  # 추가 페이지 로드 대기
        logging.info(f"이 섹션의 {nth_child - 1}번째 페이지 버튼을 성공적으로 클릭했습니다.")
    except Exception as e:
        logging.error(f"페이지 버튼 {nth_child} 클릭 중 오류 발생: {e}")
        raise

def click_next_section(driver):
    try:
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, ".pagination > .next.is_active"))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
        time.sleep(0.5)
        next_button.click()
        # 섹션 로드 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".room_item"))
        )
        time.sleep(3)  # 추가 로드 대기
        logging.info("다음 섹션으로 이동했습니다.")
    except Exception as e:
        logging.error(f"다음 섹션 버튼 클릭 중 오류 발생: {e}")
        raise

def test_pagination(driver, max_sections=100, pages_per_section=10):
    section_number = 1  # 섹션 번호 초기화

    while section_number <= max_sections:
        logging.info(f"--- 섹션 {section_number} 시작 ---")
        
        try:
            if section_number == 1:
                # 첫 번째 섹션: 페이지 2는 a:nth-child(2), 페이지 3~12는 a:nth-child(4) ~ a:nth-child(11)
                click_page_button(driver, 2)  # 페이지 2

                for nth in range(4, 12):  # 페이지 3~10
                    click_page_button(driver, nth)
            else:
                # 두 번째 섹션 이상: 페이지 12, 22, ...는 a:nth-child(3), 페이지 13~22, 23~32, ...는 a:nth-child(4) ~ a:nth-child(11)
                click_page_button(driver, 3)  # 첫 페이지 (12, 22, 32, ...)

                for nth in range(4, 12):  # 페이지 다음 9개 (2번째~10번째 페이지)
                    click_page_button(driver, nth)

            # 다음 섹션으로 이동
            click_next_section(driver)
            section_number += 1

        except Exception as e:
            logging.warning(f"섹션 {section_number} 처리 중 오류 발생: {e}")
            break  # 오류 발생 시 페이지네이션 테스트 종료

    logging.info("페이지네이션 테스트가 완료되었습니다.")

try:
    # 대상 웹사이트 열기
    target_url = "https://33m2.co.kr/"  # 실제 URL로 교체
    driver.get(target_url)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, ".room_item"))
    )
    logging.info("웹사이트에 성공적으로 접속했습니다.")
    time.sleep(2)  # 추가 페이지 로드 대기

    # 검색 수행
    keyword = "강남"  # 실제 검색 키워드로 교체
    search = driver.find_element(By.CSS_SELECTOR, '#txt_search_keyword')
    search.send_keys(keyword)
    time.sleep(0.5)

    search_btn = driver.find_element(By.CSS_SELECTOR, "#btn_search")
    search_btn.click()

    # 검색 결과 로드 대기
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, ".room_item"))
    )
    time.sleep(2)  # 추가 페이지 로드 대기

    logging.info(f"키워드 '{keyword}'에 대한 페이지네이션 테스트 시작")

    # 페이지네이션 테스트 실행
    max_sections = 100  # 테스트할 최대 섹션 수
    pages_per_section = 10  # 각 섹션에서 테스트할 최대 페이지 수

    test_pagination(driver, max_sections, pages_per_section)

except Exception as e:
    logging.error(f"웹사이트 접속 또는 검색 중 오류 발생: {e}")

finally:
    driver.quit()
